In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LogisticRegression
np.set_printoptions(suppress=True)

In [2]:
a = pickle.load(open('banknotes.data', 'rb'))
a.shape

(1372, 5)

In [3]:
p1 = a[:600,:]
p2 = a[600:900,:]
p3 = a[900:1000,:]
p4 = a[1000:1100,:]
p5 = a[1100:1150,:]
p6 = a[1150:1200,:]
test_data = a[1200:1300,:]

In [4]:
p1 = p1 - np.min(p1,0)
p1 = p1 / (np.max(p1,0)-np.min(p1,0)) 
p2 = p2 - np.min(p2,0)
p2 = p2 / (np.max(p2,0)-np.min(p2,0)) 
p3 = p3 - np.min(p3,0)
p3 = p3 / (np.max(p3,0)-np.min(p3,0)) 
p4 = p4 - np.min(p4,0)
p4 = p4 / (np.max(p4,0)-np.min(p4,0)) 
p5 = p5 - np.min(p5,0)
p5 = p5 / (np.max(p5,0)-np.min(p5,0)) 
p6 = p6 - np.min(p6,0)
p6 = p6 / (np.max(p6,0)-np.min(p6,0)) 
test_data = test_data - np.min(test_data,0)
test_data = test_data / (np.max(test_data,0)-np.min(test_data,0))

In [5]:
def test(train_data, test_data):
    train_features = train_data[:,:-1]
    train_labels = train_data[:,-1]
    test_features = test_data[:,:-1]
    test_labels = test_data[:,-1]
    
    clf = LogisticRegression()
    clf.fit(train_features, train_labels)
    predictions = clf.predict(test_features)
    
    acc = np.mean(np.abs(predictions-test_labels))
    
    return predictions,acc
v = np.array([test(p,test_data)[0] for p in [p1,p2,p3,p4,p5,p6]])
v = 2*(v-0.5)
acc = np.array([[test(p,test_data)[1] for p in [p1,p2,p3,p4,p5,p6]]])
print(acc)
regular_vote = np.mean(np.abs(((np.sum(v,0)>0)-test_data[:,-1])))
print(regular_vote)
trust = (acc-np.min(acc))/(np.max(acc)-np.min(acc))
print(trust)

[[0.02 0.04 0.19 0.13 0.22 0.22]]
0.17
[[0.   0.1  0.85 0.55 1.   1.  ]]


In [6]:
def Cosine(v,iterations,test_data):
    nv,nq = v.shape
    v2 = v*v/2
    v_2 = v/2
    ev = (np.sum(v2 + v_2, 1) + np.sum(v2 - v_2, 1))/(np.sum(2*v2-1,1)+nq)
    ev.shape = (nv, 1)
    wf = np.ones(test_data[:,-1].shape)
    F = v*wf
    eta = 0.1
    
    for it in range(iterations):
        posF = np.sum(F*(v2+v_2), 1)
        negF = np.sum(F*(v2-v_2), 1)
        zeros = (np.sum(2*v2-1, 1)+nq)
        F_ = (F*2*v2)**2
        F2 = np.sum(F_, 1)
        norm = zeros*F2
        norm = np.sqrt(norm)
        posF.shape, negF.shape, norm.shape = ev.shape, ev.shape, ev.shape

        old_ev = ev*(1-eta)
        new_ev = (posF + negF)*eta
        ev = old_ev + new_ev

        ev = ev / norm
        V = v*ev

        Vt = V.transpose()
        v2t = v2.transpose()
        v_2t = v_2.transpose()

        posV = np.sum((Vt * (v2t + v_2t))**3, 1)
        negV = np.sum((Vt * (v2t - v_2t))**3, 1)
        norm = posV - negV
        posV.shape, negV.shape, norm.shape = wf.shape, wf.shape, wf.shape
        wf = posV + negV
        wf = wf / norm
        F = v*wf
        print(np.mean(np.abs((wf>0)-test_data[:,-1])))
    return wf, ev

pred = Cosine(v,10,test_data)
trust = pred[1].transpose()
print(trust)
print(1-(trust-np.min(trust))/(np.max(trust)-np.min(trust)))

0.2
0.17
0.17
0.17
0.17
0.17
0.17
0.17
0.17
0.17
[[0.08460078 0.08787383 0.09476862 0.09487556 0.09009944 0.09104394]]
[[1.         0.68144803 0.01040862 0.         0.46483915 0.37291507]]


In [7]:
def est3(v,iterations,test_data):
    nq, nv = v.shape
    
    trust = 0.1*np.ones(nv) #voters trustworthiness
    diff = 0.1*np.ones(nq) #queries difficulty
    y = np.ones(nq)

    for it in range(iterations):
        
        for i in range(nq):
            n = np.sum(v[i]!=0)
            pos = np.sum((v[i]==1)*(1-trust*diff[i]))
            neg = np.sum((v[i]==-1)*trust*diff[i])
            y[i] = (pos+neg)/n
        
        for i in range(nq):
            n = np.sum(v[i]!=0)
            pos = np.sum((v[i]==1)*(1-y[i])/trust)
            neg = np.sum((v[i]==-1)*y[i]/trust)
            if n!=0:
                diff[i] = (pos+neg)/n

        for i in range(nv):
            n = np.sum(v.transpose()[i]!=0)
            pos = np.sum((v.transpose()[i]==1)*(1-y)/diff)
            neg = np.sum((v.transpose()[i]==-1)*y/diff)
            if n!=0:
                trust[i] = (pos+neg)/n
        print(np.mean(np.abs((np.array(y)>0.5)-test_data[:,-1])))
    return np.array(y),np.array(trust),np.array(diff)

pred = est3(v.transpose(),10, test_data)
trust = pred[1]
print(trust)
print((trust-np.min(trust))/(np.max(trust)-np.min(trust)))

0.17
0.17
0.12
0.14
0.14
0.06
0.06
0.06
0.06
0.06
[0.09990189 0.09989981 0.09990006 0.09989994 0.09990361 0.0999021 ]
[0.54793549 0.         0.06580534 0.03489616 1.         0.60256276]


In [8]:
def map_to_ring(x):
    a = 2**20
    a = a * x
    a = np.rint(a)
    a = a.astype(int)
    a = a % 2**60
    return a

def secret_share(x, real=True, modulo=False):
    if not modulo:
        modulo = 2**60
    if real:
        x = map_to_ring(x)
    shares_partner = np.random.randint(0, modulo, x.shape)
    pickle.dump(shares_partner, open('banknotes.alice', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
    shares_identity = (x - shares_partner).astype(int) % modulo
    pickle.dump(shares_identity, open('banknotes.bob', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

#secret_share(v)
#pickle.dump(a, open('banknotes.data', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
#pickle.dump(test_data[:,-1], open('banknotes.testtarget', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)